## Simple Streamlit App for Cortex LLM

Simple app showing off ability to do completion, sentiment, summarization, question/ansswe, and translation with Cortex LLM.

In [ ]:
# Import python packages
import streamlit as st
from streamlit import cache_resource, cache_data
from snowflake.snowpark.session import Session
from snowflake.snowpark.context import get_active_session
import re
import json

session = get_active_session()

@cache_data(show_spinner='CALLING LLM')
def complete(model, prompt):
    global session
    prompt = re.sub("'", "''", prompt)
    sql_code = f"SELECT SNOWFLAKE.CORTEX.COMPLETE('{model}', '{prompt}') as answer"
    df = session.sql(sql_code).to_pandas()
    return df['ANSWER'][0], sql_code

@cache_data(show_spinner='CALLING LLM')
def get_sentiment(text):
    global session
    text = re.sub("'", "''", text)
    sql_code = f"SELECT SNOWFLAKE.CORTEX.SENTIMENT('{text}') as answer"
    df = session.sql(sql_code).to_pandas()
    return df['ANSWER'][0], sql_code

@cache_data(show_spinner='CALLING LLM')
def summarize(text):
    global session
    text = re.sub("'", "''", text)
    sql_code = f"SELECT SNOWFLAKE.CORTEX.SUMMARIZE('{text}') as answer"
    df = session.sql(sql_code).to_pandas()
    return df['ANSWER'][0], sql_code


@cache_data(show_spinner='CALLING LLM')
def extract_answer(from_text, question):
    global session
    from_text = re.sub("'", "''", from_text)
    question =  re.sub("'", "''", question)
    sql_code = f"SELECT SNOWFLAKE.CORTEX.EXTRACT_ANSWER('{from_text}','{question}') as answer"
    #st.write (sql_code)
    
    df = session.sql(sql_code).to_pandas()
    return df['ANSWER'][0], sql_code


@cache_data(show_spinner='CALLING LLM')
def translate(from_lang, to_lang, text):
    global session
    text = re.sub("'", "''", text)
    sql_code = f"SELECT SNOWFLAKE.CORTEX.TRANSLATE('{text}', '{from_lang}', '{to_lang}') as answer"
    df = session.sql(sql_code).to_pandas()
    return df['ANSWER'][0], sql_code


tab1, tab2,tab3,tab4, tab5 = st.tabs(['Completion', 'Sentiment','Summarize', 'Translate', 'Extract Answer'])


with tab1:

    st.header("Completion")
    st.subheader("Ask the LLM anything")
    completion_input = st.text_area("Prompt",height=400)
    completion_model = st.selectbox("Choose Model", options = ['llama2-70b-chat','mistral-7b'])
    if st.button("Complete"):
        if completion_input is not None and completion_model is not None:
            result, sql_code = complete(completion_model,completion_input)
            st.success(result)
            with st.expander("See SQL"):
                st.code(sql_code,language='SQL')
                
           
with tab2:

    st.header("Sentiment Analysis")
    st.subheader("Get the Sentiment of a Text")
    sentiment_input = st.text_area("Text for Sentiment Analysis",height=400)
    
    if st.button("Get Sentiment"):
        if sentiment_input is not None: 
            result, sql_code = get_sentiment(sentiment_input)
            if result > 0:
                st.success(f" Positive. The Sentiment Value is {result}")
            else:
                
                st.warning(f" Negative. The Sentiment Value is {result}")
            with st.expander("See SQL"):
                st.code(sql_code,language='SQL')


with tab3:

    st.header("Summarize")
    st.subheader("Summarize a Text")
    sentiment_input = st.text_area("Text for Summarization", height=400)
    
    if st.button("Summarize"):
        if sentiment_input is not None: 
            result, sql_code = summarize(sentiment_input)
            st.success(result)
            with st.expander("See SQL"):
                st.code(sql_code,language='SQL')

with tab4:

    st.header("Translate")
    st.subheader("Translate a Text")
    language_options = {"German":'de', "French":'fr', "Turkish":'tr', "Korean":'ko',
                "Potrtuguese":'pt', "English":'en' , "Italian": 'it', "Japanese":'ja',
                "Spanish":'es',"Russian": 'ru',"Swedish": 'sv', "Polish": 'pl' }
    language_names = list(language_options.keys())
    from_language = st.selectbox("From", options=language_names )
    if from_language is not None:
        to_language_names =  language_names.copy()
        
        to_language_names.remove(from_language)
        to_language =   st.selectbox("To", options =  to_language_names)
        translate_input = st.text_area("Text for Translation", height=400)
        from_language_code = language_options[from_language]
        
    if to_language is not None:
        to_language_code = language_options[to_language]
    
    if st.button("Translate"):
        if to_language_code is not None: 
            result, sql_code = translate(from_language_code,to_language_code,translate_input)
            st.success(result)
            with st.expander("See SQL"):
                st.code(sql_code,language='SQL')

with tab5:
    st.header("Extract Answer")
    st.subheader("Extract answer from a Piece of Text")
    input_text = st.text_area("Source Text", height=400)
    question = st.text_input("Question")
    
    if st.button("Get Answer"):
        if input_text is not None and question is not None: 
            result, sql_code = extract_answer(input_text,question)
            st.success(json.loads(result)[0]['answer'])
            with st.expander("See SQL"):
                st.code(sql_code,language='SQL')
